Process of habitat mapping: 
1. Construct 3d models (Visual SfM)
2. 3D model processing (Meshlab) (calibration for scale and orientation, meshing)  
3. Calculate terrain variables (Tutorial1)
4. Make occurrence dataset (Meshlab)   
5. Convert data format for habitat suitability modelling (Tutorial2)
6. Habitat suitability modelling (MaxEnt)  
7. Habitat mapping on TIN model (Tutorial3)

See documents details about Process 1, 2, 4, 6

ハビタットマッピング実施のプロセス
1. 3次元モデルの構築 (Visual SfM)
2. 3次元モデルの編集 (Meshlab) (スケール・方位の校正やメッシュ化など)
3. 地形条件の計算 (Tutorial 1での解説内容)
4. 生物の出現データの生成(Meshlab)
5. MaxEntに適用するためのデータ形式の変更(Tutorial2)
6. MaxEntでの生息適地モデリング
7. TINモデル上に予測図を作成する(Tutorial2)

Tutorialでは，3, 5, 7の内容を実行します．1, 2, 4, 6は別のソフトウェアを用いて作業を行う必要があります．
これらの詳細なやり方は，habmaptoolsデータセット準備マニュアルにおいて解説しています．
本コードの最も簡単な動かし方は，リポジトリ全体をダウンロードし，habmaptools.pyと同じディレクトリ（tutorial1.ipynbも同じ階層に含む)上で
tutorialのpathやデータを置き換えて用いることです．

Tutorial1: read ply file and calculate terrain variables

Recommended1: run block1 and block2  
Recommended2: run block1 and block2 (comment out line 21), next run Tutorial1B

tutorial 1では，セル[1]とセル[2]を実行すれば，とりあえず地形条件が計算できます．

In [1]:
#import libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import habmaptools

以下のセルに示したメインプロセスのみを動かせば，最も簡単に全ての地形条件を取得できる． 
マニュアルに倣って，3次元点群モデルとメッシュモデルを用意してください．
path_file, path_output, model_fileの指定を用意したモデル，お使いの環境に合うように設定すればよい．
ただし，さらに高速に動作するJuliaによる計算プログラムや異なる計算範囲の地形条件を計算するツールを公開しています．  
Juliaコードを用いる場合には，#Main01ブロックまで実行してください．

In [2]:
#Main Process01
#pathを正しく設定すれば，以下のコードで地形条件計算までの機能を行えます．
#Juliaで地形条件を計算する場合には，最終行をコメントアウトしてください．
path_file=os.getcwd()+"/sample/sample_model/"
path_output=os.getcwd()+"/sample/sample_output/"
kernel_size=[0.1, 0.2, 0.4, 0.8]

model_file=["modelA01"]#, "modelR01"]
for model_index in model_file:
    path_mesh = path_file+model_index+"_mesh.ply"
    path_vert = path_file+model_index+"_vert.ply"
    path_data=os.getcwd()+"/sample/sample_output/"+model_index+"/"
    os.makedirs(path_data, exist_ok=True)
    
    #01 read ply model
    habmaptools.main_mesh(path_mesh, path_data)
    habmaptools.main_raw(path_vert, path_data)
    
    #02 calculate terrain variables (default set >comment on next cell)
    p1, v1, f1 = habmaptools.ply_read(path_file+model_index+"_mesh.ply")
    print(len(v1),len(f1)) #the length of vertice and face
    path_data=os.getcwd()+"/sample/sample_output/"+model_index+"/ply_parts/"
    
    #stop_point番目までの地形条件を計算する．本番ではstop_point=-1に変更してください．
    #habmaptools.main_calc_variables(model_index, path_data, path_output, kernel_size, stop_point=-1)
    %time habmaptools.main_calc_variables(model_index, path_data, path_output, kernel_size, stop_point=100)

13427 23077
Wall time: 9.85 s


計算した地形条件をplyファイルにマッピングする．

現在のプログラムでは20000面を対象に地形条件を計算するのに30分ほど時間がかかります．  
Juliaプログラムでは5倍ほど高速で計算できますのでそちらの使用を推奨します．  
Pythonのほうももう少し早くなるはずですが，主目的ではないので取り組む暇がないでいます．  

本tutorialで説明している機能一覧です．
01A

In [2]:
#01A: read ply files and get property, vertice, faces dataset.
#メッシュモデルのplyファイルを読み込み，プロパティ・頂点・面情報を取得します．
model_index="modelA01"
path_file=os.getcwd()+"/sample/sample_model/"
p1, v1, f1 = habmaptools.ply_read(path_file+model_index+"_mesh.ply") #引数はメッシュモデルのパスです．

#頂点と面の最初2つの情報を出力します．
print(v1[0:2]) #head of vertice datset
print(f1[0:2]) #head of faces dataset

[['-2.912616' '-0.1831709' '-1.14842' '-0.9602985' '0.105398' '0.2582982'
  '255' '255' '255' '255' '\n']
 ['0.1401883' '-0.02865968' '-0.7065747' '-0.467076' '-0.222323'
  '0.855811' '255' '255' '255' '255' '\n']]
[['3' '4295' '12411' '12750' '255' '255' '255' '255' '\n']
 ['3' '5153' '6197' '7763' '255' '255' '255' '255' '\n']]


In [23]:
#01B: read ply files (vertice only) and get property, vertice dataset.
#点群モデルのplyファイルを読み込み，プロパティ・頂点情報を取得します．01Aと同じ関数を用いるので，
p2, v2, f2 = habmaptools.ply_read(path_file+model_index+"_vert.ply") #引数は点群モデルのパスです．

#頂点の最初2つの情報を出力します．面はないので，何も出力されません．
print(v2[0:2]) #head of vertice dataset
print(f2) #faces dataset (Null)

[['-0.2250313' '-0.4003885' '-1.135942' '-0.2775664' '-0.003079371'
  '0.9607015' '\n']
 ['-1.913142' '-0.4798024' '-0.005322363' '0.1240797' '0.3498206'
  '0.9285633' '\n']]
[]


In [9]:
#02A: save the csv file of the vertice coordinates(vertice_dataset.csv), 
#vertice normal(vertice_normal.csv), faces (faces_dataaset.csv),
#faces colors(faces_color.csv), property(property.csv), minimum property(length of vertices and faces)
path_output=os.getcwd()+"/sample/sample_output/"
habmaptools.write_csv(p1, v1, f1, path_output)

In [ ]:
#02B: save the csv file of the vertice coordinates(vertice_dataset_vert.csv), 
#vertice normal(vertice_normal_vert.csv)
path_output=os.getcwd()+"/sample/sample_output/"
vertice_dataset2=pd.DataFrame(v2)
vertice_dataset2.iloc[:,0:3].to_csv(path_output+'ply_parts/vertice_dataset_vert.csv',header=False, index=False)
vertice_dataset2.iloc[:,3:6].to_csv(path_output+'ply_parts/vertice_normal_vert.csv',header=False, index=False)

In [14]:
#Func03: Analysis of mesh model
d, S=habmaptools.analysis_mesh(v1, f1) #from Func01A
print("mean edge length(m):", np.mean(d))
print("mean face areas(m^2):", np.mean(S))

mean edge length(m): 0.05113148385328753
mean face areas(m^2): 0.0007684296161937915


In [ ]:
#Main01: merge Func01A~Func02B
model_file=["modelA01", "modelR01"]
for model_index in model_file:
    model_path = path_file+model_index+".ply"
    os.makedirs("/ply_parts", exist_ok=True)
    habmaptools.main_mesh(model_path, path_output)
    habmaptools.main_raw(model_path, path_output)